In [102]:
import numpy as np
from scipy.optimize import minimize
from honeycomb_analysis import youngs_modulus_E1, youngs_modulus_E2, shear_modulus_G12

# Fixed length of the cell
l = 6.0

# Define weight coefficients
lambda_1 = 0.33
lambda_2 = 0.33
lambda_3 = 0.34

# Ensure the sum of weights is 1
assert lambda_1 + lambda_2 + lambda_3 == 1.0, "The weights should sum to 1."

def objective(H):
    alpha_l, beta_l, theta_deg = H
    try:
        E_x_star = youngs_modulus_E1(beta_l, l, alpha_l, theta_deg)
        E_y_star = youngs_modulus_E2(beta_l, l, alpha_l, theta_deg)
        G_xy_star = shear_modulus_G12(beta_l, l, alpha_l, theta_deg)
        
        if np.any(np.isnan([E_x_star, E_y_star, G_xy_star])) or np.any(np.isinf([E_x_star, E_y_star, G_xy_star])):
            return np.inf  # Return a large value if any modulus is invalid
        
        # Normalize the values to avoid extreme numbers
        E_x_star /= 1e9
        E_y_star /= 1e9
        G_xy_star /= 1e9
        
        return -(lambda_1 * E_x_star + lambda_2 * E_y_star + lambda_3 * G_xy_star)  # Negative for maximization since the optimizer is a minimizer
    except Exception as e:
        return np.inf

# Constraints
def constraint_alpha_lower(H):
    alpha_l, beta_l, theta_deg = H
    alpha = alpha_l / l
    return alpha - 0.5

def constraint_alpha_upper(H):
    alpha_l, beta_l, theta_deg = H
    alpha = alpha_l / l
    return 1 - alpha

def constraint_beta_lower(H):
    alpha_l, beta_l, theta_deg = H
    beta = beta_l / l
    return beta - 0.004

def constraint_beta_upper(H):
    alpha_l, beta_l, theta_deg = H
    beta = beta_l / l
    return 0.008 - beta

def constraint_theta_lower(H):
    alpha_l, beta_l, theta_deg = H
    return theta_deg + 30

def constraint_theta_upper(H):
    alpha_l, beta_l, theta_deg = H
    return 30 - theta_deg

def calculate_relative_density(alpha_l, beta_l, theta_deg):
    t = beta_l
    return (2 / np.sqrt(3)) * (t / l) * (1 - (t / (2 * np.sqrt(3) * l)))

def constraint_relative_density(H, rho_ori):
    alpha_l, beta_l, theta_deg = H
    rho_s_star = calculate_relative_density(alpha_l, beta_l, theta_deg)
    return rho_ori - rho_s_star

# Modulus constraints
def constraint_modulus_E1_upper(H):
    alpha_l, beta_l, theta_deg = H
    E1 = youngs_modulus_E1(beta_l, l, alpha_l, theta_deg)
    return 1e9 - E1

def constraint_modulus_E2_upper(H):
    alpha_l, beta_l, theta_deg = H
    E2 = youngs_modulus_E2(beta_l, l, alpha_l, theta_deg)
    return 1e9 - E2

def constraint_modulus_G12_upper(H):
    alpha_l, beta_l, theta_deg = H
    G12 = shear_modulus_G12(beta_l, l, alpha_l, theta_deg)
    return 1e9 - G12

# Optimization function
def run_optimization(initial_alpha_l, initial_beta_l, initial_theta, rho_ori):
    try:
        bounds = [(3.0, 12.0), (0.024, 0.048), (-30, 30)]
        x0 = [initial_alpha_l, initial_beta_l, initial_theta]

        constraints = [
            {'type': 'ineq', 'fun': constraint_alpha_lower},
            {'type': 'ineq', 'fun': constraint_alpha_upper},
            {'type': 'ineq', 'fun': constraint_beta_lower},
            {'type': 'ineq', 'fun': constraint_beta_upper},
            {'type': 'ineq', 'fun': constraint_theta_lower},
            {'type': 'ineq', 'fun': constraint_theta_upper},
            {'type': 'ineq', 'fun': lambda H: constraint_relative_density(H, rho_ori)},
            {'type': 'ineq', 'fun': constraint_modulus_E1_upper},
            {'type': 'ineq', 'fun': constraint_modulus_E2_upper},
            {'type': 'ineq', 'fun': constraint_modulus_G12_upper}
        ]

        # Validate initial guess
        initial_constraints = [constraint['fun'](x0) for constraint in constraints]

        if any(c < 0 for c in initial_constraints):
            raise ValueError("Initial guess does not satisfy the constraints")

        # Use SLSQP for optimization
        solution_slsqp = minimize(objective, x0, method='SLSQP', bounds=bounds, constraints=constraints, options={'disp': False})

        results = ""
        if solution_slsqp.success:
            slsqp_solution = f"SLSQP Solution: {solution_slsqp.x}\nSLSQP Objective value: {-solution_slsqp.fun}\n"
            slsqp_density = calculate_relative_density(*solution_slsqp.x)
            slsqp_solution += f"SLSQP Optimized Density: {slsqp_density}\n"
            results += slsqp_solution
        else:
            slsqp_solution = f"SLSQP Optimization failed: {solution_slsqp.message}\n"
            results += slsqp_solution

        # Use COBYLA for optimization
        solution_cobyla = minimize(objective, x0, method='COBYLA', constraints=constraints, options={'disp': False})

        if solution_cobyla.success:
            cobyla_solution = f"COBYLA Solution: {solution_cobyla.x}\nCOBYLA Objective value: {-solution_cobyla.fun}\n"
            cobyla_density = calculate_relative_density(*solution_cobyla.x)
            cobyla_solution += f"COBYLA Optimized Density: {cobyla_density}\n"
            results += cobyla_solution
        else:
            cobyla_solution = f"COBYLA Optimization failed: {solution_cobyla.message}\n"
            results += cobyla_solution

        return results

    except Exception as e:
        return str(e)

# Example usage
initial_alpha_l = 6.0  # Initial guess for alpha_l
initial_beta_l = 0.04  # Initial guess for beta_l
initial_theta = 30.0   # Initial guess for theta
rho_ori = 0.0153       # Original relative density

# Run the optimization
results = run_optimization(initial_alpha_l, initial_beta_l, initial_theta, rho_ori)
print(results)




SLSQP Solution: [ 6.    0.04 30.  ]
SLSQP Objective value: 6.887147211133138e-09
SLSQP Optimized Density: 0.007683188774380197
COBYLA Solution: [5.95905894e+00 4.80000000e-02 4.31443848e-03]
COBYLA Objective value: 0.3299999985081441
COBYLA Optimized Density: 0.009216270973700681

